# GeoProgrammierung 2

## Uebung 01

### Aufgabe 1: SwissALTI3D

In [1]:
import os
from urllib.request import urlopen

def download(url, destfile, overwrite=True):
    print("Downloading", destfile, "from", url)

    if os.path.exists(destfile) and not overwrite:
        print("File already exists, not overwriting.")
        return
    
    response = urlopen(url) 
    info = response.info()
    cl = info["Content-Length"]
    
    if cl != None:
        filesize = int(cl)
        currentsize = 0
        
        with open(destfile, 'wb') as f:
            while True:
                chunk = response.read(16*1024)
                currentsize += len(chunk)
                
                if not chunk:
                    break
                f.write(chunk)
                percent = int(100*currentsize/filesize)
                
                bar = "*"*(percent)
                bar += "-"*((100-percent))
                print('\r{}% done \t[{}]'.format(percent, bar), end='')
        print("")
        
    else:
        print("Downloading please wait... (filesize unknown)")
        with open(destfile, 'wb') as f:
            while True:
                chunk = response.read(16*1024)
                if not chunk:
                    break
                f.write(chunk)

In [2]:
from zipfile import ZipFile

def unzip(source, dest):
    with ZipFile(source, 'r') as zz:
        #zz.extractall(path=dest) We don't overwrite already extracted files:
        for item in zz.infolist():
                file_path = os.path.join(dest, item.filename)
                if not os.path.exists(file_path):
                    zz.extract(item, dest)

Diese Utility-Funktionen sind auch in der Datei „download.py“ verfügbar. In Zukunft importieren wir sie einfach mit:

    from download import download,unzip

### Download

Zuerst erstellen wir ein neues Verzeichnis Uebung01, falls es noch nicht existiert.

In [3]:
if not os.path.exists("Uebung01"):
    os.mkdir("Uebung01")

Direkte Download-Links sind:

SwissALTI3D Links:
* https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2023_2769-1184/swissalti3d_2023_2769-1184_0.5_2056_5728.tif
* https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2023_2769-1183/swissalti3d_2023_2769-1183_0.5_2056_5728.tif
* https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2023_2770-1184/swissalti3d_2023_2770-1184_0.5_2056_5728.tif
* https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2023_2770-1183/swissalti3d_2023_2770-1183_0.5_2056_5728.tif
* https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2023_2771-1184/swissalti3d_2023_2771-1184_0.5_2056_5728.tif
* https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2023_2771-1183/swissalti3d_2023_2771-1183_0.5_2056_5728.tif


In [5]:
dest_dir = "Uebung01/swissalti3d/"

if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)

base = " https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2023_"

urls = [base + "2769-1184/swissalti3d_2023_2769-1184_0.5_2056_5728.tif", base + "2769-1183/swissalti3d_2023_2769-1183_0.5_2056_5728.tif", base + "2770-1184/swissalti3d_2023_2770-1184_0.5_2056_5728.tif", base + "2770-1183/swissalti3d_2023_2770-1183_0.5_2056_5728.tif",
        base + "2771-1184/swissalti3d_2023_2771-1184_0.5_2056_5728.tif", base + "2771-1183/swissalti3d_2023_2771-1183_0.5_2056_5728.tif"]

for url in urls:
    download(url, dest_dir + os.path.basename(url), False)

100% done 	[****************************************************************************************************]
100% done 	[****************************************************************************************************]
100% done 	[****************************************************************************************************]
100% done 	[****************************************************************************************************]
100% done 	[****************************************************************************************************]
100% done 	[****************************************************************************************************]


In [8]:
!gdalinfo Uebung01/swissalti3d/swissalti3d_2023_2769-1183_0.5_2056_5728.tif

Driver: GTiff/GeoTIFF
Files: Uebung01/swissalti3d/swissalti3d_2023_2769-1183_0.5_2056_5728.tif
Size is 2000, 2000
Coordinate System is:
PROJCRS["CH1903+ / LV95",
    BASEGEOGCRS["CH1903+",
        DATUM["CH1903+",
            ELLIPSOID["Bessel 1841",6377397.155,299.1528128,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4150]],
    CONVERSION["Swiss Oblique Mercator 1995",
        METHOD["Hotine Oblique Mercator (variant B)",
            ID["EPSG",9815]],
        PARAMETER["Latitude of projection centre",46.9524055555556,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8811]],
        PARAMETER["Longitude of projection centre",7.43958333333333,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8812]],
        PARAMETER["Azimuth at projection centre",90,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8813]],
        P

### a) Erstellen eines VRT

Das GDAL Virtual Format (VRT) ist eine einfache Möglichkeit, einen neuen Datensatz aus Kacheln zu erstellen. Es handelt sich grundsätzlich um eine Textdatei (XML), aber die VRT-Datei kann so behandelt werden, als wäre sie eine einzelne Rasterdatei.

In [9]:
!gdalbuildvrt Uebung01/swissalti3d/swissalti3d.vrt Uebung01/swissalti3d/swissalti3d_2023_2769-1184_0.5_2056_5728.tif Uebung01/swissalti3d/swissalti3d_2023_2769-1183_0.5_2056_5728.tif Uebung01/swissalti3d/swissalti3d_2023_2770-1184_0.5_2056_5728.tif Uebung01/swissalti3d/swissalti3d_2023_2770-1183_0.5_2056_5728.tif Uebung01/swissalti3d/swissalti3d_2023_2771-1184_0.5_2056_5728.tif Uebung01/swissalti3d/swissalti3d_2023_2771-1183_0.5_2056_5728.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [10]:
!gdalinfo Uebung01/swissalti3d/swissalti3d.vrt

Driver: VRT/Virtual Raster
Files: Uebung01/swissalti3d/swissalti3d.vrt
       Uebung01/swissalti3d\swissalti3d_2023_2769-1184_0.5_2056_5728.tif
       Uebung01/swissalti3d\swissalti3d_2023_2769-1183_0.5_2056_5728.tif
       Uebung01/swissalti3d\swissalti3d_2023_2770-1184_0.5_2056_5728.tif
       Uebung01/swissalti3d\swissalti3d_2023_2770-1183_0.5_2056_5728.tif
       Uebung01/swissalti3d\swissalti3d_2023_2771-1184_0.5_2056_5728.tif
       Uebung01/swissalti3d\swissalti3d_2023_2771-1183_0.5_2056_5728.tif
Size is 6000, 4000
Coordinate System is:
PROJCRS["CH1903+ / LV95",
    BASEGEOGCRS["CH1903+",
        DATUM["CH1903+",
            ELLIPSOID["Bessel 1841",6377397.155,299.1528128,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4150]],
    CONVERSION["unnamed",
        METHOD["Hotine Oblique Mercator (variant B)",
            ID["EPSG",9815]],
        PARAMETER["Latitude of projection centre",4

### a) Overviews (Pyramidenstufen) hinzufügen

Wenn Sie versuchen, diese Datei in QGIS zu öffnen, ist sie viel zu langsam und benötigt auch extrem viel Speicher.

Wir müssen mehrere Auflösungen dieser Datei erstellen, die **Overlays** (auch **Pyramide** genannt) genannt werden.
Dies kann mit dem Tool „gdaladdo“ erfolgen.

Mit dem Parameter -r kann die Resampling-Methode angegeben werden. Es kann sein:

nearest, average, rms, bilinear, gauss, cubic, cubicspline, lanczos, average_magphase, mode

Nach dem Dateinamen müssen wir Ebenen als Zweierpotenz angeben, zum Beispiel 2 4 8 16 32

Die Bearbeitung wird sehr lange dauern. Wenn Sie dies mit einem sehr grossen Datensatz (>1 TB) versuchen, kann es Tage bis hin zu Wochen dauern. Auch in diesem Fall empfiehlt es sich, den Speicher zu vergrößern und die Parallelverarbeitung zu ermöglichen. Dies kann mit diesem Konfigurationsflag erfolgen:


`--config GDAL_NUM_THREADS ALL_CPUS` (oder ganzzahliger Wert, wenn Sie die Anzahl der CPUs angeben möchten)

`--config GDAL_CACHEMAX 8000` (Cache in MB, je nach RAM so hoch wie möglich)       

In [11]:
!gdaladdo -r average Uebung01/swissalti3d/swissalti3d.vrt 2 4 8 16 32

0...10...20...30...40...50...60...70...80...90...100 - done.


### b) Erstellte Datensätze im QGIS anschauen

### c) Geländeschummerung erstellen mit den gdal-utilities

In [14]:
!gdaldem hillshade Uebung01/swissalti3d/swissalti3d.vrt Uebung01/swissalti3d/hillshade.png

0...10...20...30...40...50...60...70...80...90...100 - done.


In [17]:
# Create a color ramp file:
#
# Elevation RGB color

color_text_file = """0 110 110 110
1600 120 120 120
1800 140 140 140
2000 160 160 160
2200 180 180 180
2400 200 200 200
2600 220 220 220
2800 230 230 230
3000 250 250 250"""

file = open("Uebung01/swissalti3d/color2.txt", "w")
file.write(color_text_file)
file.close()

In [18]:
!gdaldem color-relief Uebung01/swissalti3d/swissalti3d.vrt Uebung01/swissalti3d/color2.txt Uebung01/swissalti3d/hillshade_color2.png

0...10...20...30...40...50...60...70...80...90...100 - done.


### d) Vektorfile mit Höhenlinien mit den gdal-utilities erstellen

In [20]:
!gdal_contour -a elevation -i 250 Uebung01/swissalti3d/swissalti3d.vrt Uebung01/swissalti3d/hoehenlinien.shp

0...10...20...30...40...50...60...70...80...90...100 - done.


### Aufgabe 2: Swissimage

Direkte Download-Links sind:

Swissimage Links:
* https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2022_2769-1182/swissimage-dop10_2022_2769-1182_0.1_2056.tif
* https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2022_2769-1183/swissimage-dop10_2022_2769-1183_0.1_2056.tif

### a) Erstellen eines VRT und Overlays

In [21]:
dest_dir = "Uebung01/swissimage/"

if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)

base = "https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2022_2769-"

urls = [base + "1182/swissimage-dop10_2022_2769-1182_0.1_2056.tif", base + "1183/swissimage-dop10_2022_2769-1183_0.1_2056.tif"]

for url in urls:
    download(url, dest_dir + os.path.basename(url), False)

100% done 	[****************************************************************************************************]
100% done 	[****************************************************************************************************]


In [22]:
!gdalbuildvrt Uebung01/swissimage/swissimage.vrt Uebung01/swissimage/swissimage-dop10_2022_2769-1182_0.1_2056.tif Uebung01/swissimage/swissimage-dop10_2022_2769-1183_0.1_2056.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [23]:
!gdaladdo -r average Uebung01/swissimage/swissimage.vrt 2 4 8 16 32

0...10...20...30...40...50...60...70...80...90...100 - done.


### b) Konvertierung in ein einziges GeoTiff

In [24]:
!gdal_translate Uebung01/swissimage/swissimage.vrt Uebung01/swissimage/swissimage_merged.tif

Input file size is 10000, 20000
0...10...20...30...40...50...60...70...80...90...100 - done.
